# Using Mappings as Inputs

`DeeplayModule`objects support mapping objects (container objects that support arbitrary key lookups) as input, allowing users to define complex data processing pipelines in a simple and intuitive way.

## Example

To use mappings as input to `DeeplayModule`objects, the user must define the mapping between the input and output data structures. This is done by configuring the `.set_input_map()` and `.set_output_map()` methods of the module.

For example, this code defines a linear module that takes a dictionary with a key `x` as input and overwrites the key `x` with the output of the linear transformation.

In [1]:
import deeplay as dl
import torch

module = dl.Layer(torch.nn.Linear, 10, 64)

module.set_input_map("x")
module.set_output_map("x")

Layer[Linear](in_features=10, out_features=64)

Currently supported mapping objects include dictionaries, such as ... 

In [2]:
### add dictionary example?

... and Torch Geometric Data objects, such as ...

In [3]:
from torch_geometric.data import Data

input_data = Data(x=torch.randn(100, 10))

Once the module is built, the input data can be passed to the module as with `Tensor` objects. The module will automatically map the input and output data to the expected data structure.

In [4]:
built = module.build()

output_data = built(input_data)

print("output keys: ", output_data.keys)
print("input x shape: ", input_data.x.shape)
print("output x shape: ", output_data.x.shape)

output keys:  <bound method BaseData.keys of Data(x=[100, 64])>
input x shape:  torch.Size([100, 10])
output x shape:  torch.Size([100, 64])


## Basic Operations with Mappings

The `.set_input_map()` method supports local key re-assigment, useful when the input data key doesn't match the expected key by the module. For example:

In [5]:
module = dl.Layer(torch.nn.MultiheadAttention, embed_dim=10, num_heads=2)

module.set_input_map(query="x", key="x", value="x")

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

Additionally, the `.set_output_map()` method receives a second argument to specify the mapping for the attention weights output ...

In [6]:
module.set_output_map("x", "attention_weights")

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

... or alternatively ...

In [7]:
module.set_output_map(x=0, attention_weights=1)

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

where, `x=0` and `attention_weights=1` indicate that `x` and `attention_weights` in the output dictionary correspond to the first and second outputs of the module, respectively.

In [8]:
built = module.build()

input_data = Data(x=torch.randn(100, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys)
print("output x shape: ", input_data.x.shape)
print("output attention_weights shape: ", output_data.attention_weights.shape)

output keys:  <bound method BaseData.keys of Data(x=[100, 10], attention_weights=[100, 100])>
output x shape:  torch.Size([100, 10])
output attention_weights shape:  torch.Size([100, 100])


New keys can be added to the output data structure to store intermediate results.

In [9]:
module = dl.Layer(torch.nn.MultiheadAttention, embed_dim=10, num_heads=2)

module.set_input_map(query="x", key="x", value="x")

module.set_output_map(
    x=0,
    intermediate_x=0,
    attention_weights=1,
    intermediate_attention_weights=1,
)

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

In [10]:
built = module.build()

input_data = Data(x=torch.randn(100, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys)

print("output x shape: ", input_data.x.shape)
print("output intermediate_x shape: ", output_data.intermediate_x.shape)

print("output attention_weights shape: ", output_data.attention_weights.shape)
print(
    "output intermediate_attention_weights shape: ",
    output_data.intermediate_attention_weights.shape,
)

output keys:  <bound method BaseData.keys of Data(x=[100, 10], intermediate_x=[100, 10], attention_weights=[100, 100], intermediate_attention_weights=[100, 100])>
output x shape:  torch.Size([100, 10])
output intermediate_x shape:  torch.Size([100, 10])
output attention_weights shape:  torch.Size([100, 100])
output intermediate_attention_weights shape:  torch.Size([100, 100])


## Mappings Allow Sequential Processing

Mapping enables the sequencing of branched pipelines, fully exploiting the modular design of DeeplayModules. For instance:

In [11]:
from torch_geometric.nn import LayerNorm

module = dl.Sequential(
    dl.Layer(torch.nn.Bilinear, 10, 10, 10).set_input_map("x1", "x2") \
        .set_output_map("y"),
    dl.Layer(LayerNorm, 10).set_input_map(x="y", batch="batch") \
        .set_output_map("y"),
    dl.Add().set_input_map("x1", "y").set_output_map("x1"),
    dl.Add().set_input_map("x2", "y").set_output_map("x2"),
)

In the code above:

1. First, a Bilinear layer combines inputs `x1` and `x2`, and the result is labeled as `y`. 

2. Then, a layer norm operation is applied to `y`, considering batch information from `batch`, and the output is newly assigned to `y`. 

3. Finally, `x1` and `x2` are independently summed with `y` and stored in the keys `x1` and `x2`, respectively, implementing a residual connection.


In [12]:
built = module.build()

input_data = Data(
    x1=torch.randn(5, 10),
    x2=torch.randn(5, 10),
    batch=torch.Tensor([0, 0, 0, 1, 1]).long(),
)

output_data = built(input_data)

print("output keys: ", output_data.keys)
print("output x1 shape: ", output_data.x1.shape)
print("output x2 shape: ", output_data.x2.shape)
print("output y shape: ", output_data.y.shape)

output keys:  <bound method BaseData.keys of Data(x1=[5, 10], x2=[5, 10], batch=[5], y=[5, 10])>
output x1 shape:  torch.Size([5, 10])
output x2 shape:  torch.Size([5, 10])
output y shape:  torch.Size([5, 10])


## From Mappings to Tensor Objects

Deeplay supports a seamless transition between mapping objects and Tensor objects. The output data structure can be converted to a Tensor object using the `FromDict` module:

In [13]:
module = dl.Sequential(
    dl.Layer(torch.nn.Bilinear, 10, 10, 10).set_input_map("x1", "x2") \
        .set_output_map("y"),
    dl.Layer(LayerNorm, 10).set_input_map(x="y", batch="batch") \
        .set_output_map("y"),
    dl.FromDict("y"), 
    dl.Layer(torch.nn.Linear, 10, 1)
)

In [14]:
built = module.build()

input_data = Data(
    x1=torch.randn(5, 10),
    x2=torch.randn(5, 10),
    batch=torch.Tensor([0, 0, 0, 1, 1]).long(),
)

output_data = built(input_data)

print("output type: ", type(output_data))
print("output shape: ", output_data.shape)

output type:  <class 'torch.Tensor'>
output shape:  torch.Size([5, 1])


## Defining Parallel Pipelines

Deeplay's `Parallel` method enables users to create parallel pipelines that can process input mapping objects concurrently. Each module in the pipeline receives the input data, extracts the relevant keys, and processes the data independently. Finally, the outputs from each module are merged into a single mapping object.

In [15]:
module = dl.Parallel(
    dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x").set_output_map("y"),
    dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x").set_output_map("z"),
)

built = module.build()

input_data = Data(x=torch.randn(5, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys)
print("output y shape: ", output_data.y.shape)
print("output z shape: ", output_data.z.shape)

output keys:  <bound method BaseData.keys of Data(x=[5, 10], y=[5, 30], z=[5, 30])>
output y shape:  torch.Size([5, 30])
output z shape:  torch.Size([5, 30])


Alternatively, the output mappings can be specified directly within `Parallel`´s constructor. 

In the example below, each module is given a key (`y` and `z` for the first and second layer, respectively), and the output mappings are automatically determined based on these keys.

```python
module = dl.Parallel(
    y=dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x"),
    z=dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x"),
)
```

This method streamlines the syntax but offers less explicit control over output mappings compared to individual module configuration.